In [1]:
import pandas as pd
from sqlalchemy import create_engine

### Extract CSVs into DataFrames

In [2]:
pokemon_file = "../Resources/pokemon.csv"
pokemon_df = pd.read_csv(pokemon_file)
pokemon_df.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


In [3]:
types_file = "../Resources/types.csv"
types_df = pd.read_csv(types_file)
types_df.head()

,id,identifier,generation_id,damage_class_id
0,1,normal,1,2.0
1,2,fighting,1,2.0
2,3,flying,1,2.0
3,4,poison,1,2.0
4,5,ground,1,2.0


In [4]:
moves_file = "../Resources/moves.csv"
moves_df = pd.read_csv(moves_file)
moves_df.head()

,id,identifier,generation_id,type_id,power,pp,accuracy,priority,target_id,damage_class_id,effect_id,effect_chance,contest_type_id,contest_effect_id,super_contest_effect_id
0,1,pound,1,1,40.0,35.0,100.0,0,10,2,1,NaN,5.0,1.0,5.0
1,2,karate-chop,1,2,50.0,25.0,100.0,0,10,2,44,NaN,5.0,2.0,5.0
2,3,double-slap,1,1,15.0,10.0,85.0,0,10,2,30,NaN,5.0,10.0,7.0
3,4,comet-punch,1,1,18.0,15.0,85.0,0,10,2,30,NaN,5.0,12.0,7.0
4,5,mega-punch,1,1,80.0,20.0,85.0,0,10,2,1,NaN,5.0,1.0,18.0


### Transform Pokemon Types DataFrame

In [5]:
# Create a filtered dataframe from specific columns for Pokemon
types_cols = ["id","identifier"]
types_transformed = types_df[types_cols].copy()

# Rename the Column Headers
types_transformed = types_transformed.rename(columns={"id": "TID","identifier": "Type"})

# Create new Index Column
types_transformed.insert(0,"id",1)
types_transformed['id'] = types_transformed.index + 1

# Set Index
types_transformed.set_index("id",inplace=True)

# Capitalize 
types_transformed['Type']=types_transformed['Type'].str.capitalize()

types_transformed.head()

,TID,Type
id,,
1,1,Normal
2,2,Fighting
3,3,Flying
4,4,Poison
5,5,Ground


### Transform Pokemon Moves DataFrame

In [6]:
# Create a filtered dataframe from specific columns for Pokemon
moves_cols = ["id","identifier","type_id"]
moves_transformed = moves_df[moves_cols].copy()

# Rename the Column Headers
moves_transformed = moves_transformed.rename(columns={"id": "MID","identifier": "Moves","type_id": "TID"})

# Set index
moves_transformed.set_index("MID", inplace=True)

# Capitalize
moves_transformed['Moves']=moves_transformed['Moves'].str.title()

moves_transformed.head()

,Moves,TID
MID,,
1,Pound,1
2,Karate-Chop,2
3,Double-Slap,1
4,Comet-Punch,1
5,Mega-Punch,1


### Transform Pokemon DataFrame

In [7]:
# Create a filtered dataframe from specific columns for Pokemon
pokemon_cols = ["Name","Type 1","Attack","Defense","Speed"]
pokemon_transformed = pokemon_df[pokemon_cols].copy()

# Rename the Column Headers
pokemon_transformed = pokemon_transformed.rename(columns={"Name": "Pokemon_Name","Type 1": "Type"})

# Create new Index Column
pokemon_transformed.insert(0,"id",1)
pokemon_transformed['id'] = pokemon_transformed.index + 1

# Set index
pokemon_transformed.set_index("id", inplace=True)

pokemon_transformed.head()


,Pokemon_Name,Type,Attack,Defense,Speed
id,,,,,
1,Bulbasaur,Grass,49,49,45
2,Ivysaur,Grass,62,63,60
3,Venusaur,Grass,82,83,80
4,VenusaurMega Venusaur,Grass,100,123,80
5,Charmander,Fire,52,43,65


### Merge Pokemon & Type Transformed Dataframes

In [8]:
# Merge on type to include typeid
pokemon_merged=pokemon_transformed.merge(types_transformed, on='Type', how='outer')

pokemon_merged_fix=pokemon_merged[['Pokemon_Name','TID','Type','Attack','Defense','Speed']].dropna()
pokemon_merged_fix.set_index("Pokemon_Name", inplace=True)
pokemon_merged_fix.head()

,TID,Type,Attack,Defense,Speed
Pokemon_Name,,,,,
Bulbasaur,12,Grass,49.0,49.0,45.0
Ivysaur,12,Grass,62.0,63.0,60.0
Venusaur,12,Grass,82.0,83.0,80.0
VenusaurMega Venusaur,12,Grass,100.0,123.0,80.0
Oddish,12,Grass,50.0,55.0,30.0


### Create database connection

In [9]:
connection_string = "<username>:<password>@localhost:5432/pokemon_db"
engine = create_engine(f'postgresql://{connection_string}')

In [10]:
# Confirm tables
engine.table_names()

['Moves', 'Types', 'Pokemon']

### Load DataFrames into database

In [11]:
types_transformed.to_sql(name='Types', con=engine, if_exists='append', index=True)

In [12]:
moves_transformed.to_sql(name='Moves', con=engine, if_exists='append', index=True)

In [13]:
pokemon_merged_fix.to_sql(name='Pokemon', con=engine, if_exists='append', index=True)